<a href="https://colab.research.google.com/github/Sohampatel26/OlivaresLab_ProjectContributions/blob/main/Mapping_top%2C_lower_and_upper_soil_data(yearly_and_quarterly).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import pandas as pd
import geopandas as gpd
import os
import matplotlib.pyplot as plt

# Mount Google Drive
drive.mount('/content/drive')

# Define the folder containing the CSV files
csv_folder_path = '/content/drive/MyDrive/sam'

# Load the California shapefile
shapefile_path = '/content/drive/MyDrive/EE ML PFAS/Raw_Data/Mapping/California_Shapefile/California.shp'
ca_shapefile = gpd.read_file(shapefile_path)
ca_shapefile = ca_shapefile.to_crs('EPSG:4326')

# Function to convert DataFrame to GeoDataFrame
def convert_to_gdf(df, lat_col='Latitude', lon_col='Longitude'):
    return gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[lon_col], df[lat_col]), crs='EPSG:4326')

# Function to plot and save GeoDataFrame as TIFF
def plot_and_save_tiff(gdf, title, output_path):
    fig, ax = plt.subplots(figsize=(10, 10))
    ca_shapefile.plot(ax=ax, color='lightgrey', edgecolor='black')
    if not gdf.empty:
        gdf.plot(ax=ax, color='blue', markersize=1)
    plt.title(title)
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.savefig(output_path, format='tiff')
    plt.close()

# Process each CSV file in the folder
for filename in os.listdir(csv_folder_path):
    if filename.endswith('.csv'):
        # Load CSV file
        file_path = os.path.join(csv_folder_path, filename)
        soil_data = pd.read_csv(file_path)

        # Convert to GeoDataFrame
        soil_gdf = convert_to_gdf(soil_data)

        # Reproject to match California shapefile CRS
        soil_gdf = soil_gdf.to_crs(ca_shapefile.crs)

        # Perform spatial intersection
        soil_ca = gpd.sjoin(soil_gdf, ca_shapefile, how='inner', op='intersects')

        # Check for invalid or missing latitude values
        invalid_latitudes = soil_ca[
            (soil_ca.geometry.y < -90) | (soil_ca.geometry.y > 90) | soil_ca.geometry.y.isna()
        ]

        # If there are invalid latitudes, print a message and continue
        if not invalid_latitudes.empty:
            print(f"Invalid or missing latitudes found in file: {filename}")
            continue

        # Define output path for TIFF file
        tiff_output_path = os.path.join('/content/drive/MyDrive/EE ML PFAS/Raster_Data/Environmental_Quality/Soil_Quality/', f'{filename.replace(".csv", "")}.tif')

        # Plot and save TIFF
        plot_and_save_tiff(soil_ca, f'Soil Data in California - {filename}', tiff_output_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate